In [1]:
import os
import glob
import SimpleITK as sitk
import numpy as np
import pandas as pd


# 设置路径
data_dir = './videos/bed_rest_MRI/bed_rest_MRI_L4L5'

label_dir = os.path.join(data_dir, 'label_in_png_L4L5_4class_DINO_order')
prompt_dir = 'manual_prompt_frame0_1_2'
prompt_dir = 'manual_prompt_frame0_1'
# prompt_dir = 'manual_prompt_frame0'

auto_seg_dir = os.path.join(data_dir, prompt_dir, 'SAM2_seg_mask_nolap')


# 创建新的目标目录
output_dir = os.path.join(data_dir, prompt_dir, 'seg_mask_renamed')
os.makedirs(output_dir, exist_ok=True)

In [2]:


# 目录设置
auto_seg_dir = os.path.join(data_dir, prompt_dir, 'seg_in_nii_72_L4-L5')    # 自动分割结果所在目录
manual_seg_dir = os.path.join(data_dir, 'label_in_nii_72_L4-L5')             # 手动分割结果所在目录

# 创建 DataFrame 用于存储结果
df_results = pd.DataFrame(columns=["PID", "Class", "Auto Area", "Manual Area"])

# 遍历自动分割结果文件
for idx, auto_filename in enumerate(os.listdir(auto_seg_dir)):
    print(f"{idx}: Processing {auto_filename}")

    # 提取 PID，假设文件名格式类似 "ID1A_R0_XXX.nii.gz"，PID 为前两段 "ID1A_R0"
    parts = auto_filename.split('_')
    pid = "_".join(parts[0:3])

    # 自动分割文件读取
    auto_path = os.path.join(auto_seg_dir, auto_filename)
    auto_img = sitk.ReadImage(auto_path)
    auto_array = sitk.GetArrayFromImage(auto_img)
    auto_spacing = auto_img.GetSpacing()  # 例如 (spacing_x, spacing_y, spacing_z)

    # 查找对应的手动分割文件（根据 PID 匹配）
    manual_files = glob.glob(os.path.join(manual_seg_dir, pid + "*.nii.gz"))
    if not manual_files:
        print(f"Warning: Manual segmentation file not found for {pid}")
        continue
    manual_path = manual_files[0]
    manual_img = sitk.ReadImage(manual_path)
    manual_array = sitk.GetArrayFromImage(manual_img)
    manual_spacing = manual_img.GetSpacing()

    # 对于每个类别（例如 1,2,3,4）计算面积（单位：原面积除以 100，并保留 3 位小数）
    for class_value in [1, 2, 3, 4]:
        # 自动分割中该类别的像素数及面积
        auto_mask = (auto_array == class_value)
        auto_pixel_count = np.sum(auto_mask)
        auto_area = auto_pixel_count * auto_spacing[0] * auto_spacing[1]
        auto_area = round(auto_area / 100, 3)

        # 手动分割中该类别的像素数及面积
        manual_mask = (manual_array == class_value)
        manual_pixel_count = np.sum(manual_mask)
        manual_area = manual_pixel_count * manual_spacing[0] * manual_spacing[1]
        manual_area = round(manual_area / 100, 3)

        # 将结果追加到 DataFrame 中
        df_results = df_results._append({
            "PID": pid,
            "Class": class_value,
            "Auto Area": auto_area,
            "Manual Area": manual_area
        }, ignore_index=True)

# 根据 prompt_dir 过滤掉指定前缀的 PID
manual_prompt_id = ['ID4D_BDC', 'ID12M_R13', 'ID15Q_R13']
# 逻辑：根据 prompt_dir 中是否包含 "1" 或 "2" 来决定去除 manual_prompt_id 中的前几项
remove_prefixes = manual_prompt_id[: 1 + ("1" in prompt_dir) + ("2" in prompt_dir)]
print("Remove prefixes:", remove_prefixes)
filter_pid = lambda df: df[~df["PID"].astype(str).str.startswith(tuple(remove_prefixes))]
df_results = filter_pid(df_results)


# 保存 CSV 文件
csv_save_path = os.path.join(data_dir, prompt_dir, prompt_dir + "_area_auto_vs_manual.csv")
df_results.to_csv(csv_save_path, index=False)
print(f"Results saved to {csv_save_path}")


0: Processing ID9J_R0_L4-L5_sam2_seg_slice23_seg.nii.gz
1: Processing ID24Z_BDC_L4-L5_sam2_seg_slice15_seg.nii.gz
2: Processing ID10K_BDC_L4-L5_sam2_seg_slice42_seg.nii.gz
3: Processing ID12M_BDC_L4-L5_sam2_seg_slice13_seg.nii.gz


/tmp/ipykernel_17570/4116111359.py:47: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_results = df_results._append({


4: Processing ID22X_R13_L4-L5_sam2_seg_slice22_seg.nii.gz
5: Processing ID17S_BDC_L4-L5_sam2_seg_slice20_seg.nii.gz
6: Processing ID7G_R13_L4-L5_sam2_seg_slice12_seg.nii.gz
7: Processing ID15Q_R0_L4-L5_sam2_seg_slice22_seg.nii.gz
8: Processing ID8H_R0_L4-L5_sam2_seg_slice24_seg.nii.gz
9: Processing ID14P_R0_L4-L5_sam2_seg_slice21_seg.nii.gz
10: Processing ID14P_R13_L4-L5_sam2_seg_slice24_seg.nii.gz
11: Processing ID6F_BDC_L4-L5_sam2_seg_slice17_seg.nii.gz
12: Processing ID19U_R0_L4-L5_sam2_seg_slice22_seg.nii.gz
13: Processing ID13N_R0_L4-L5_sam2_seg_slice35_seg.nii.gz
14: Processing ID23Y_R0_L4-L5_sam2_seg_slice11_seg.nii.gz
15: Processing ID1A_R0_L4-L5_sam2_seg_slice17_seg.nii.gz
16: Processing ID5E_R0_L4-L5_sam2_seg_slice19_seg.nii.gz
17: Processing ID20V_R13_L4-L5_sam2_seg_slice25_seg.nii.gz
18: Processing ID6F_R13_L4-L5_sam2_seg_slice10_seg.nii.gz
19: Processing ID17S_R0_L4-L5_sam2_seg_slice23_seg.nii.gz
20: Processing ID9J_R13_L4-L5_sam2_seg_slice39_seg.nii.gz
21: Processing ID12

In [3]:
print(df_results.shape)
df_results.head()



(280, 4)


PID Class  Auto Area  Manual Area
0    ID9J_R0_L4-L5     1     15.116       14.421
1    ID9J_R0_L4-L5     2     10.482       11.329
2    ID9J_R0_L4-L5     3     11.687       11.014
3    ID9J_R0_L4-L5     4     15.648       14.443
4  ID24Z_BDC_L4-L5     1      5.013        7.053

In [4]:
import numpy as np
from scipy.stats import ks_1samp, norm
from statsmodels.stats.weightstats import ttost_paired
from tqdm import tqdm

# 类别名称映射
class_names = {
    1: "Right_LES",
    2: "Right_MF",
    3: "Left_MF",
    4: "Left_LES"
}
# 自定义打印顺序
print_order = [4, 1, 3, 2]

# 参数
margin_pct = 0.05
n_bootstrap = 10000

# 用于记录每类的最小margin和对应p值
min_margins = []

for class_value in print_order:
    label_name = class_names[class_value]
    df_class = df_results[df_results["Class"] == class_value]
    if df_class.empty:
        continue

    label = df_class["Manual Area"].values
    auto = df_class["Auto Area"].values
    min_len = min(len(label), len(auto))
    label, auto = label[:min_len], auto[:min_len]
    diff = auto - label

    # 正态性检验
    z = (diff - np.mean(diff)) / np.std(diff, ddof=1)
    ks_stat, ks_p = ks_1samp(z, norm.cdf)
    delta = np.mean(label) * margin_pct

    print(f"==== Class: {label_name} ====")
    print(f"Sample size: {min_len}")
    print(f"[KS normality] p = {ks_p:.4f}")

    if ks_p >= 0.05:
        # parametric TOST
        p_tost, *_ = ttost_paired(auto, label, -delta, delta)
        print("→ Using parametric TOST")
        print(f"pTOST = {p_tost:.4f} {'✅ Equivalent' if p_tost < 0.05 else '❌ Not equivalent'}")

        # 总是执行 margin 搜索
        for m in np.arange(0.01, 0.501, 0.001):
            d = np.mean(label) * m
            p_equiv, *_ = ttost_paired(auto, label, -d, d)
            if p_equiv < 0.05:
                min_margins.append((m * 100, p_equiv))
                print(f"→ Minimum margin for equivalence: δ = ±{m*100:.2f}% at p value {p_equiv:.3f}\n")
                break
        else:
            min_margins.append((np.nan, np.nan))
            print("→ Even δ = ±50% is not enough for equivalence\n")

    else:
        # bootstrap TOST
        inside = 0
        for _ in tqdm(range(n_bootstrap), leave=False):
            idx = np.random.choice(len(diff), len(diff), replace=True)
            mean_diff = np.mean(diff[idx])
            if -delta < mean_diff < delta:
                inside += 1
        p_boot = 1 - (inside / n_bootstrap)
        print("→ Using bootstrap TOST")
        print(f"bootstrap pTOST = {p_boot:.4f} {'✅ Equivalent' if p_boot < 0.05 else '❌ Not equivalent'}")

        # 总是执行 margin 搜索
        for m in np.arange(0.01, 0.501, 0.001):
            d = np.mean(label) * m
            inside_test = 0
            for _ in range(10000):
                idx = np.random.choice(len(diff), len(diff), replace=True)
                mean_diff = np.mean(diff[idx])
                if -d < mean_diff < d:
                    inside_test += 1
            p_equiv = 1 - (inside_test / 10000)
            if p_equiv < 0.05:
                min_margins.append((m * 100, p_equiv))
                print(f"→ Minimum margin for equivalence: δ = ±{m*100:.2f}%, at p value {p_equiv:.3f}\n")
                break
        else:
            min_margins.append((np.nan, np.nan))
            print("→ Even δ = ±50% is not enough for equivalence\n")

# 最后统一打印
print("### Minimum Margin Summary ###")
for margin, pval in min_margins:
    if not np.isnan(margin):
        print(f"{margin:.2f}% ({pval:.3f})")
    else:
        print("N/A (no equivalence within 50%)")


==== Class: Left_LES ====
Sample size: 70
[KS normality] p = 0.0091


→ Using bootstrap TOST
bootstrap pTOST = 0.0358 ✅ Equivalent


→ Minimum margin for equivalence: δ = ±4.70%, at p value 0.048

==== Class: Right_LES ====
Sample size: 70
[KS normality] p = 0.0342


→ Using bootstrap TOST
bootstrap pTOST = 0.5070 ❌ Not equivalent


→ Minimum margin for equivalence: δ = ±8.90%, at p value 0.045

==== Class: Left_MF ====
Sample size: 70
[KS normality] p = 0.6260
→ Using parametric TOST
pTOST = 0.9992 ❌ Not equivalent
→ Minimum margin for equivalence: δ = ±13.30% at p value 0.047

==== Class: Right_MF ====
Sample size: 70
[KS normality] p = 0.0429


→ Using bootstrap TOST
bootstrap pTOST = 0.2059 ❌ Not equivalent


→ Minimum margin for equivalence: δ = ±7.20%, at p value 0.047

### Minimum Margin Summary ###
4.70% (0.048)
8.90% (0.045)
13.30% (0.047)
7.20% (0.047)


In [5]:
import pingouin as pg

# 初始化
epsilon = 1e-6
mape_results = []
icc_results = []

# 分析每一类
for class_value in print_order:
    label_name = class_names[class_value]
    df_class = df_results[df_results["Class"] == class_value]
    if df_class.empty:
        continue

    label = df_class["Manual Area"].values
    auto = df_class["Auto Area"].values
    min_len = min(len(label), len(auto))
    label, auto = label[:min_len], auto[:min_len]

    # MAPE
    ape = np.abs((auto - label) / (label + epsilon))
    mape = np.mean(ape)
    mape_std = np.std(ape)
    mape_results.append([label_name, mape * 100, mape_std * 100])

    # ICC
    df_long = df_class.melt(id_vars=["PID"], value_vars=["Manual Area", "Auto Area"],
                            var_name="Method", value_name="Area")
    icc_df = pg.intraclass_corr(data=df_long, targets="PID", raters="Method", ratings="Area")
    icc_row = icc_df[icc_df["Type"] == "ICC2"].iloc[0]
    icc_value = icc_row["ICC"]
    ci_lower, ci_upper = icc_row.get("CI95%") if "CI95%" in icc_row else (np.nan, np.nan)
    icc_results.append([label_name, icc_value, ci_lower, ci_upper])

# 输出 MAPE
print("\n### MAPE Results ###")
for cls, m, s in mape_results:
    print(f"{cls}: {m:.1f} ± {s:.1f}%")

# 输出 ICC
print("\n### ICC Results ###")
for cls, icc, lo, hi in icc_results:
    print(f"{cls}: {icc:.3f}, [{lo:.2f} - {hi:.2f}]")



### MAPE Results ###
Left_LES: 12.7 ± 13.3%
Right_LES: 14.4 ± 12.7%
Left_MF: 14.5 ± 13.8%
Right_MF: 14.3 ± 21.4%

### ICC Results ###
Left_LES: 0.762, [0.64 - 0.85]
Right_LES: 0.794, [0.68 - 0.87]
Left_MF: 0.727, [0.35 - 0.87]
Right_MF: 0.722, [0.59 - 0.82]
